In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import  f1_score
from charformer_pytorch import GBST

In [ ]:
!pip install charformer-pytorch

In [2]:
import random
import numpy as np
import torch 
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
def eval_conversation(text):
    input_ids = torch.tensor([list(text.encode("utf-8"))]) + 3 # add 3 for special tokens
    return input_ids


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class MetNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.dim = 512
        self.tokenizer = GBST(
            num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
            dim = 512,                    # dimension of token and intra-block positional embedding
            max_block_size = 4,           # maximum block size
            downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
            score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
        ).to(device)
        self.positionEmbeddings = nn.Embedding(512 ,64)
        self.transformerLayer = nn.TransformerEncoderLayer(576,8) 
        self.linear1 = nn.Linear(576,  64)
        self.linear2 = nn.Linear(64,  1)
        self.linear3 = nn.Linear(512,  16)
        self.linear4 = nn.Linear(16,  1)

    def forward(self, x):
        x = x.type(torch.LongTensor).to(device)
        tokenize = self.tokenizer(x)
        positions = (torch.arange(0,self.dim ).reshape(1,self.dim ) + torch.zeros(x.shape[0],self.dim )).to(device)
        sentence = torch.cat((tokenize[0],self.positionEmbeddings(positions.long())),axis=2)
        attended = self.transformerLayer(sentence)
        linear1 = F.relu(self.linear1(attended))
        linear2 = F.relu(self.linear2(linear1))
        linear2 = linear2.view(-1,512) # reshaping the layer as the transformer outputs a 2d tensor (or 3d considering the batch size)
        linear3 = F.relu(self.linear3(linear2))
        out = torch.sigmoid(self.linear4(linear3))
        return out
    

In [6]:
model = MetNet().to(device)

In [8]:
path_to_json_file = '/scratch/sm9669/sarcasm_detection_shared_task_twitter_training.jsonl'
import json 

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

In [9]:
from tqdm import tqdm
accurate = 0
tot_tweets = 0

tweets=[]
labels=[]
for tweet in tqdm(json_data):
    tot_cont = ""
    for con in tweet['context']:
        tot_cont+=con 
    tot_cont+=tweet['response']
    tweets.append(tot_cont)
    labels.append(tweet['label'])

100%|██████████| 5000/5000 [00:00<00:00, 436025.53it/s]


In [10]:
import pandas as pd
label_list=[]
for i in range(0,len(labels)):
    if labels[i]=="SARCASM":
        label_list.append(1)
    else:
        label_list.append(0)

X_train =tweets
y_train = label_list

In [11]:
train_data=[]
mi=10000000000000000
ma=-1
for i in range(0,len(X_train)):
    train_data.append(eval_conversation(X_train[i]));
    mi=min(train_data[i].shape[1],mi)
    ma=max(train_data[i].shape[1],ma)

new_train_data=[]
for i in range(0,len(train_data)):
    if train_data[i].shape[1]<2048:
        z = torch.zeros(1, 2048-train_data[i].shape[1])
        s = torch.cat((train_data[i],z),1)
        new_train_data.append(s)
    else:
        cur_train_data = torch.reshape(train_data[i][0][:2048], (1, 2048)).type(torch.FloatTensor)
        new_train_data.append(torch.Tensor(cur_train_data))

new_train_data = torch.stack(new_train_data)
print(new_train_data.shape)

y_train=torch.Tensor(y_train)
print(y_train.shape)
new_train_data = torch.squeeze(new_train_data)


torch.Size([5000, 1, 2048])
torch.Size([5000])


In [12]:
import torch.optim as  optim

In [13]:
import torch.utils.data as data_utils
train=data_utils.TensorDataset(new_train_data ,y_train)


train_loader = torch.utils.data.DataLoader(train, batch_size=32,
               shuffle=True, num_workers=0, pin_memory=True)

In [14]:
path_to_json_file = '/scratch/sm9669/sarcasm_detection_shared_task_twitter_testing.jsonl'
import json 

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

In [15]:
from tqdm import tqdm

tweets=[]
labels=[]
for tweet in tqdm(json_data):
    tot_cont = ""
    for con in tweet['context']:
        tot_cont+=con 
    tot_cont+=tweet['response']
    tweets.append(tot_cont)
    labels.append(tweet['label'])

100%|██████████| 1800/1800 [00:00<00:00, 421867.86it/s]


In [16]:
import pandas as pd
test_label_list=[]
for i in range(0,len(labels)):
    if labels[i]=="SARCASM":
        test_label_list.append(1)
    else:
        test_label_list.append(0)

X_test =tweets
y_test = test_label_list

In [17]:
test_data=[]
mi=10000000000000000
ma=-1
for i in range(0,len(X_test)):
    test_data.append(eval_conversation(X_test[i]));
    mi=min(test_data[i].shape[1],mi)
    ma=max(test_data[i].shape[1],ma)

print(mi, ma)
new_test_data=[]
for i in range(0,len(test_data)):
    if test_data[i].shape[1]<2048:
        z = torch.zeros(1, 2048-test_data[i].shape[1])
        s = torch.cat((test_data[i],z),1)
        new_test_data.append(s)
    else:
        cur_test_data = torch.reshape(test_data[i][0][:2048], (1, 2048)).type(torch.FloatTensor)
        new_test_data.append(torch.Tensor(cur_test_data))

new_test_data = torch.stack(new_test_data)
print(new_test_data.shape)

y_test=torch.Tensor(y_test)
print(y_test.shape)
new_test_data = torch.squeeze(new_test_data)


117 4140
torch.Size([1800, 1, 2048])
torch.Size([1800])


In [18]:
import torch.utils.data as data_utils
test=data_utils.TensorDataset(new_test_data ,y_test)


test_loader = torch.utils.data.DataLoader(test, batch_size=32,
               shuffle=True, num_workers=0, pin_memory=True)

In [19]:
lr =  0.0045
optimizer = torch.optim.SGD(model.parameters(), lr,
                            momentum=0.9,
                            weight_decay=1e-4)

In [20]:
def calculateMetrics(ypred,ytrue):
    acc  = accuracy_score(ytrue,ypred)
    f1  = f1_score(ytrue,ypred)
    f1_average  = f1_score(ytrue,ypred,average="macro")
    return " f1 score: "+str(round(f1,3))+" f1 average: "+str(round(f1_average,3))+" accuracy: "+str(round(acc,3))

In [21]:
def train(train_loader, model, optimizer, epoch):

    model.train()
    trainpreds = torch.tensor([])
    traintrues = torch.tensor([])
    testpreds = torch.tensor([])
    testtrues = torch.tensor([])
    for i, (input, target) in enumerate(train_loader):
        input_var = input.cuda()
        target_var = target.cuda()
        optimizer.zero_grad()
        output = model(input_var)
        output = torch.squeeze(output, 1)
        trainpreds = torch.cat((trainpreds,output.cpu().detach()))
        traintrues = torch.cat((traintrues,target_var.cpu().detach()))
        
        loss = F.binary_cross_entropy(output,target_var)
        loss.backward()
        optimizer.step()
        
    err = F.binary_cross_entropy(trainpreds,traintrues)    
    print("train BCE loss: ",err.item(),calculateMetrics(torch.round(trainpreds).numpy(),traintrues.numpy()))
    
    model.eval()
    for i, (input, target) in enumerate(test_loader):
        input_var = input.cuda()
        target_var = target.cuda()
        output = model(input_var)
        output = torch.squeeze(output, 1)

        testpreds = torch.cat((testpreds,output.cpu().detach()))
        testtrues = torch.cat((testtrues,target_var.cpu().detach()))

        loss = F.binary_cross_entropy(output,target_var)

    print("test BCE loss: ",calculateMetrics(torch.round(testpreds).numpy(),testtrues.numpy()))


In [22]:
from tqdm import tqdm
for epoch in tqdm(range(0, 30)):
    lr = lr * (0.1 ** (epoch // 30))
    train(train_loader, model, optimizer, epoch)

  0%|          | 0/30 [00:00<?, ?it/s]

train BCE loss:  0.6935937404632568  f1 score: 0.373 f1 average: 0.485 accuracy: 0.509


  3%|▎         | 1/30 [00:21<10:22, 21.48s/it]

test BCE loss:   f1 score: 0.667 f1 average: 0.333 accuracy: 0.5
train BCE loss:  0.6913658976554871  f1 score: 0.446 f1 average: 0.518 accuracy: 0.529


  7%|▋         | 2/30 [00:42<10:01, 21.49s/it]

...

test BCE loss:   f1 score: 0.539 f1 average: 0.564 accuracy: 0.566
train BCE loss:  0.3838340640068054  f1 score: 0.83 f1 average: 0.828 accuracy: 0.828


 97%|█████████▋| 29/30 [10:28<00:21, 21.69s/it]

test BCE loss:   f1 score: 0.589 f1 average: 0.573 accuracy: 0.574
train BCE loss:  0.3666486442089081  f1 score: 0.836 f1 average: 0.835 accuracy: 0.835


100%|██████████| 30/30 [10:50<00:00, 21.68s/it]

test BCE loss:   f1 score: 0.52 f1 average: 0.564 accuracy: 0.568


In [23]:
torch.save(model.state_dict(), "charformer-twitter-30.pt")